In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
np.random.seed(42)

import numpy as np
import pandas as pd
from numpy.linalg import lstsq, cholesky
from scipy.linalg import sqrtm, schur, block_diag
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from ClassBSplines import BSpline
from TensorProductSplines import TensorProductSpline
from PenaltyMatrices import PenaltyMatrix
from Smooth import Smooths as s
 
from ClassBSplines import BSpline as b
from TensorProductSplines import TensorProductSpline as t
from DiagnosticPlot import DiagnosticPlotter
from Model_Notebook import Model, bar_chart_of_coefficient_difference_dataframe
from Model_Notebook import line_chart_of_coefficient_dataframe, check_constraint

In [55]:

#########################################################################
## DATA GENERATION
#########################################################################
np.random.seed(42)
#X = pd.DataFrame(data={"x1": np.linspace(-2,2,2000) } ) #, 
X = pd.DataFrame(data={"x1": np.logspace(0.001,0.99,1000) } ) #, 
                       
                       #"x2": np.linspace(0,1,1000), #}) #,
                       #"x3": np.linspace(-2,2,1000), "x4": np.linspace(0,1,1000) })
                       #"x5": np.linspace(-2,2,1000), "x6": np.linspace(0,1,1000)})
y = pd.DataFrame(data={"y": np.sin(25*X["x1"]) + np.square(X["x1"]) + np.random.randn(len(X["x1"]))})  # - 2*X["x2"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=True)

X_test = X_test.sort_index()
X_train = X_train.sort_index()
y_train = y_train.sort_index()
y_test = y_test.sort_index()

X_train.shape

X_norm = MinMaxScaler().fit_transform(X_train)
y_norm = StandardScaler().fit_transform(y_train)
y_norm = y_train.values

fig = make_subplots(rows=1, cols=2, 
                    specs=[[{'type':'scatter'}, {'type':'scatter3d'}] ])

fig.add_trace(go.Scatter(x=X_norm[:,0], y=y_norm.ravel(), mode="markers", name="Data"), row=1, col=1)
#fig.add_trace(go.Scatter3d(x=X_norm[:,0], y=X_norm[:,1], z=y_norm.ravel(),
#                           name="Data", mode="markers"), row=1, col=2)
fig.show()

In [58]:
######################################################
######################################################
# TEST CASE
######################################################
######################################################
m = ( ('s(1)', 'inc', 34), ) 
     #('s(2)', 'conv', 10),
     #('s(3)', 'smooth', 10), ('s(4)', 'smooth', 10), )
     #   ('s(3)', 'smooth', 10), ('s(4)', 'smooth', 10)  )

M = Model(m)
f = M.fit(X_norm, y_norm, lam_c=100, plot_=True)
bar_chart_of_coefficient_difference_dataframe(M.df_beta)
line_chart_of_coefficient_dataframe(M.df_beta)


Type x_data:  <class 'numpy.ndarray'>
Use 'x_basis' for the spline basis!
Mean squared error on data for unconstrained LS fit: 1.3209
Create basis with penalty and weight
Least squares fit iteration  1
Mean squared error on data for unconstrained LS fit: 1.3811

 Violated constraints:  6
Create basis with penalty and weight
Least squares fit iteration  2
Mean squared error on data for unconstrained LS fit: 1.3825

 Violated constraints:  8
Create basis with penalty and weight
Least squares fit iteration  3
Mean squared error on data for unconstrained LS fit: 1.3836
Iteration converged!
Mean squared error on the data: 1.3836
